# Load necessary packages

In [1]:
import pandas as pd
import numpy as np

hex_salmon = '#F68F83'
hex_gold = '#BC9661'
hex_indigo = '#2D2E5F'
hex_maroon = '#8C4750'
hex_white = '#FAFAFA'
hex_blue = '#7EB5D2'

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.dates import DateFormatter
import matplotlib.dates as dates

import matplotlib.font_manager as font_manager
mpl.font_manager._rebuild()

mpl.rcParams['font.family'] = 'SF Mono'
mpl.rcParams['font.weight'] = 'medium'
mpl.rcParams['axes.titleweight'] = 'semibold'
mpl.rcParams['axes.labelweight'] = 'medium'
mpl.rcParams['axes.prop_cycle'] = mpl.cycler(color=[hex_indigo, hex_salmon, hex_maroon])
mpl.rcParams["figure.titlesize"] = 'large'
mpl.rcParams["figure.titleweight"] = 'semibold'

from termcolor import colored

from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression, Ridge, ElasticNet, LassoCV, RidgeCV, ElasticNetCV
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score, accuracy_score

import tensorflow as tf

from entsoe import EntsoePandasClient

In [2]:
features = pd.read_pickle(f"./features.pkl")

In [3]:
features.head(5)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2015-01-01 05:00:00+00:00,27.500000,29.0,27.41,8291.75,9994.25,1702.50,25.625000,29.100000,26.357143,26.073529,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 06:00:00+00:00,26.780822,28.0,27.52,8638.25,11882.00,3243.75,29.100000,26.357143,26.073529,27.538462,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 07:00:00+00:00,25.852273,65.0,26.80,8927.25,13240.00,4312.75,26.357143,26.073529,27.538462,31.480519,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 08:00:00+00:00,24.400000,18.0,28.48,9312.50,13817.25,4504.75,26.073529,27.538462,31.480519,36.581111,...,0,0,0,0,0,0,0,0,0,0
2015-01-01 09:00:00+00:00,25.625000,15.0,27.71,9933.25,14072.50,4139.25,27.538462,31.480519,36.581111,35.371102,...,0,0,0,0,0,0,0,0,0,0


In [4]:
features.tail(5)

,ID3,VOL,MCP,LOAD,LOAD_F,LOAD_FE,ID3 (-4),ID3 (-5),ID3 (-6),ID3 (-7),...,HOD 14,HOD 15,HOD 16,HOD 17,HOD 18,HOD 19,HOD 20,HOD 21,HOD 22,HOD 23
2018-12-21 23:00:00+00:00,50.911996,499.5,50.63,11756.25,12950.00,1193.75,53.277543,53.479330,54.977402,57.227163,...,0,0,0,0,0,0,0,0,0,1
2018-12-22 00:00:00+00:00,53.554378,309.1,53.07,11164.25,12318.00,1153.75,53.479330,54.977402,57.227163,57.961460,...,0,0,0,0,0,0,0,0,0,0
2018-12-22 01:00:00+00:00,53.716854,438.8,48.50,10796.75,11741.50,944.75,54.977402,57.227163,57.961460,58.304843,...,0,0,0,0,0,0,0,0,0,0
2018-12-22 02:00:00+00:00,52.950763,294.6,46.54,10626.25,11420.75,794.50,57.227163,57.961460,58.304843,57.755109,...,0,0,0,0,0,0,0,0,0,0
2018-12-22 03:00:00+00:00,53.277543,439.2,39.50,10685.50,11313.00,627.50,57.961460,58.304843,57.755109,57.734450,...,0,0,0,0,0,0,0,0,0,0


In [5]:
# X = features.drop(labels=['ID3', 'VOL', 'LOAD_F', 'LOAD_FE'], axis=1)
# y = features['ID3']

X = features[['ID3 (-4)', 'ID3 (-5)']]
y = features['ID3']


X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size = 0.3,
    random_state = 0,
    shuffle = True)

X_train.shape, X_test.shape

((23691, 2), (10154, 2))

In [6]:
X_train_unscaled = X_train
X_test_unscaled = X_test

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train_unscaled)

X_test = scaler.transform(X_test_unscaled)

In [9]:
from ngboost import NGBRegressor

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

ngb = NGBRegressor().fit(X_train, y_train)
Y_preds = ngb.predict(X_test)
Y_dists = ngb.pred_dist(X_test)

# test Mean Squared Error
test_MSE = mean_squared_error(Y_preds, y_test)
print('Test MSE', test_MSE)

# test Negative Log Likelihood
test_NLL = -Y_dists.logpdf(y_test).mean()
print('Test NLL', test_NLL)

[iter 0] loss=4.2673 val_loss=0.0000 scale=1.0000 norm=12.6854
[iter 100] loss=3.9302 val_loss=0.0000 scale=2.0000 norm=18.6067
[iter 200] loss=3.8762 val_loss=0.0000 scale=2.0000 norm=18.1878
[iter 300] loss=3.8642 val_loss=0.0000 scale=1.0000 norm=9.0581
[iter 400] loss=3.8578 val_loss=0.0000 scale=1.0000 norm=9.0352
Test MSE 199.5160724918859
Test NLL 3.9942989655581034


In [ ]:
def MLP_model(X_train, y_train, X_test):
    # create a model
    from keras.models import Sequential
    from keras.layers import Dense
    from keras.optimizers import SGD, Adam
    
    model_MLP = Sequential()
    model_MLP.add(Dense(200, activation='relu', kernel_initializer='he_normal', input_shape=(X_train.shape[1],)))
    model_MLP.add(Dense(100, activation='relu', kernel_initializer='he_normal'))
    model_MLP.add(Dense(1))

    optimizer = Adam(clipvalue=0.5)
    # model_lstm.compile(loss = 'mse', optimizer=optimizer)

    # Compiling
    model_MLP.compile(optimizer=optimizer,loss='mean_squared_error')
    # Fitting to the training set
    model_MLP.fit(X_train, y_train, epochs = 50, batch_size = 16, verbose = 5)

    prediction = model_MLP.predict(X_test)
    # prediction = sc.inverse_transform(prediction)

    # self.model = model_lstm

    return model_MLP, prediction

model, prediction = MLP_model(X_train, y_train, X_test)

In [ ]:
# from tensorflow.keras import Sequential
# from tensorflow.keras.layers import Dense, Dropout
# from tensorflow.keras.callbacks import EarlyStopping
# #Step1. Define the model
# model = Sequential()
# model.add(Dense(16, activation = 'relu', kernel_initializer = 'he_normal', input_shape = (X_train.shape[1],)))
# model.add(Dense(8, activation = 'relu', kernel_initializer = 'he_normal'))
# model.add(Dense(1))
# #Step2. Compile the model
# model.compile(optimizer = 'adam', loss = 'mse', metrics = 'mae')
# #Step3. Fit the model
# history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,batch_size=64, verbose=5)
# #Step4.1 Evaluate the model
# loss, mae = model.evaluate(X_test, y_test)
# #Step4.2 Plot the learning curve
# plt.plot(history.history['loss'], label='train')
# plt.plot(history.history['val_loss'], label='val')
# plt.show()

# prediction = model.predict(X_test)

In [ ]:
actual_pred = pd.DataFrame()

actual_pred['Actual'] = features['ID3'][0:len(prediction)]
actual_pred['Predicted'] = prediction[:, 0]

from keras.metrics import MeanSquaredError
m = MeanSquaredError()
actual_pred['Actual'] = actual_pred['Actual'].astype(int)
actual_pred['Predicted'] = actual_pred['Predicted'].astype(int)
m.update_state(np.array(actual_pred['Actual']), np.array(actual_pred['Predicted']))

x, y = actual_pred.index, actual_pred['Actual']

fig, ax = plt.subplots(figsize = (5 ,5))
ax.plot(x, y)

plt.xticks(rotation = 90)

x, y = actual_pred.index, actual_pred['Predicted']

fig, ax = plt.subplots(figsize = (5 ,5))
ax.plot(x, y)

plt.xticks(rotation = 90);

m

In [ ]:
####################################################################################################
# Pre

actual_pred = pd.DataFrame()

actual_pred['Actual'] = features['ID3'][0:len(prediction)]
actual_pred['Predicted'] = prediction[:, 0]

####################################################################################################
# Plot 1

fig, ax = plt.subplots(figsize = (10 ,5))

x, y = actual_pred.index, actual_pred['Actual']

ax.plot(x, y)

####################################################################################################
# Plot 2

x, y = actual_pred.index, actual_pred['Predicted']

ax.plot(x, y, alpha = 0.5)

plt.xticks(rotation = 90);

In [ ]:
def smape(A, F):
    return 100/len(A) * np.sum(2 * np.abs(F - A) / (np.abs(A) + np.abs(F)))

smape(actual_pred['Actual'], actual_pred['Predicted'])